Deliverable 1: Perform ETL on Amazon Product Reviews

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-12 17:15:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-02-12 17:15:06 (6.56 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [ ]:
# from pyspark.sql.functions import to_date
# # Load in a sql function to use columns
# from pyspark.sql.functions import col
# # Read in the Review dataset as a DataFrame
# df1 = df.select(col("review_date"), to_date(col('review_date'), "yyyy-MM-dd").alias('date'))
# df1.show()

In [ ]:
# from pyspark.sql.functions import to_date
# # Read in the Review dataset as a DataFrame
# df1 = df.select(to_date(df.review_date, 'yyyy-MM-dd').alias('date'))
# df1.show()

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = df.withColumn('date', to_date(df.review_date, 'yyyy-MM-dd'))
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|      date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 b

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": 'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Deliverable 2: Determine Bias of Vine Reviews 

In [40]:
# Load in a sql function to use columns
from pyspark.sql.functions import round, col

#filter data where "total_votes" is equal to or greater than 20
vine_filtered_df = vine_df.filter(col('total_votes') >= 20)
vine_filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [53]:
#create new dataframe where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_filtered0_df = vine_filtered_df.withColumn("votes_percent", col('helpful_votes') / col('total_votes')*100)
vine_filtered1_df = vine_filtered0_df.filter(col("votes_percent") >= 50,)
vine_filtered1_df = vine_filtered1_df.withColumn('votes_percent', round('votes_percent',2))
vine_filtered1_df.show()

#check by asc
# vine_filtered1_df.orderBy(vine_filtered1_df['votes_percent'].asc()).show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        61.76|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|         60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|         84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        91.67|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        78.46|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        86.11|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        55.88|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|   

In [54]:
#create dataframe where Vine program (paid), vine == 'Y'
vine_paid_df = vine_filtered1_df.filter(col('vine') == 'Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|        88.89|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|        100.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|        79.17|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|        84.62|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|        76.92|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|         90.2|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|        78.57|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|   

In [57]:
#create dataframe where Vine program (unpaid), vine == 'N'
vine_unpaid_df = vine_filtered1_df.filter(col('vine') == 'N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        61.76|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|         60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|         84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        91.67|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        78.46|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        86.11|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        55.88|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|   

Vine Paid Reviews

In [60]:
#vine_paid The total number of reviews
vine_paid_df.count()

94

In [67]:
#create dataframe where Vine program (paid), 5-stars reviews
vine_paid_5_star_df = vine_paid_df.filter(col('star_rating') == '5')
vine_paid_5_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|        88.89|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|        79.17|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|         90.2|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|        78.57|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|         87.5|
|R3KAW29CJ8L6DQ|          5|           17|         20|   Y|                N|         85.0|
|R1OWK33OPI45KT|          5|           55|         58|   Y|                N|        94.83|
|R2UP7VTED8O425|          5|           35|         41|   Y|                N|   

In [66]:
#vine paid number of 5-star reviews
vine_paid_5_star_df.count()

48

In [65]:
#vine paid percentage 5-star reviews
paid_5_star_percent = vine_paid_5_star_df.count() / vine_paid_df.count() *100
paid_5_star_percent

51.06382978723404

Non-Vine Reviews

In [68]:
#vine_unpaid The total number of reviews
vine_unpaid_df.count()

40471

In [69]:
#create dataframe where Vine program (unpaid), 5-stars reviews
vine_unpaid_5_star_df = vine_unpaid_df.filter(col('star_rating') == '5')
vine_unpaid_5_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|votes_percent|
+--------------+-----------+-------------+-----------+----+-----------------+-------------+
|R36O341WWXXKNP|          5|           28|         31|   N|                N|        90.32|
|R29BOS5HMAY1LO|          5|           88|        110|   N|                N|         80.0|
| RDX2ZZ46AM343|          5|           32|         34|   N|                Y|        94.12|
| R60GI4Z1CNGGV|          5|           17|         20|   N|                Y|         85.0|
|R2FAARI3JQO9XQ|          5|           29|         30|   N|                Y|        96.67|
|R2WS53BH47WUMT|          5|           68|         84|   N|                N|        80.95|
|R2BBH73BUJYBGK|          5|           18|         22|   N|                N|        81.82|
|R18ZBSSAMJOV8I|          5|          450|        471|   N|                N|   

In [70]:
#unpaid number of 5-star reviews
vine_unpaid_5_star_df.count()

15663

In [71]:
#unpaid percentage 5-star reviews
unpaid_5_star_percent = vine_unpaid_5_star_df.count() / vine_unpaid_df.count() *100
unpaid_5_star_percent

38.701786464381904